In [1]:
import os
# import cosima_cookbook as cc
# from dask.distributed import Client, LocalCluster

import xarray as xr

from oceanpy import define_grid

# from gsw import f, SA_from_SP, p_from_z, geo_strf_dyn_height, grav
from gsw import f
from oceanpy import relative_vorticity, vortex_stretching, adv_relative_vorticity, adv_planetary_vorticity, vorticity_tendency # , bottom_pressure_torque, stress_curl

import numpy as np
from numbers import Number
from scipy.ndimage import uniform_filter#, gaussian_filter


In [2]:
outdir = os.path.join(os.sep, 'g', 'data', 'v45', 'jm6603', 'checkouts', 'phd', 'src', 'cosima', '02_manuscript', 'output')

In [3]:
def to_netcdf(ds, file_name):

    valid_types = (str, Number, np.ndarray, np.number, list, tuple)
    try:
        ds.to_netcdf(file_name)
    except TypeError as e:
        print(e.__class__.__name__, e)
        for variable in ds.variables.values():
            for k, v in variable.attrs.items():
                if not isinstance(v, valid_types) or isinstance(v, bool):
                    variable.attrs[k] = str(v)
        ds.to_netcdf(file_name)

## Load data

### Time of interest

In [4]:
meander_period = slice('1997-02-15', '1997-05-31')
monthly_period = slice('1997-04-01', '1997-04-30')
flex_period = slice('1997-04-10', '1997-04-25')

### Load datasets

In [5]:
coordinates = xr.open_dataset(os.path.join(outdir, 'coordinates.nc'))
hydro = xr.open_dataset(os.path.join(outdir, 'hydro.nc'))
vel = xr.open_dataset(os.path.join(outdir, 'vel.nc'))
vel_sel = vel.sel(time=monthly_period)
geos_vel = xr.open_dataset(os.path.join(outdir, 'geos-vel.nc'))

## Constants

In [6]:
# g = 9.81
rho_0 = 1036 # kg/m^3
p_ref = 0 #1500
p_mld = 200
p_int = 2500

## Define grid

In [7]:
# define coordinates
coords = {'xt_ocean': None, 'yt_ocean': None, 'st_ocean': None, 'xu_ocean': 0.5, 'yu_ocean': 0.5, 'sw_ocean': -0.5}
distances=('dxt', 'dyt', 'dst', 'dxu', 'dyu')
areas=('area_u', 'area_t')
dims=('X', 'Y', 'S')

# define grid
grid = define_grid(vel, dims, coords, distances, areas, periodic=False)

## Calculate quasi-geostrophic vorticity terms

- QG-theory: https://www.whoi.edu/fileserver.do?id=9387&pt=2&p=12288
- Closing budget in MOM6: https://mom6-analysiscookbook.readthedocs.io/en/latest/notebooks/Closing_vorticity_budget.html

Quasi-geostrophic balance

$$\frac{\partial\zeta}{\partial t} + {\bf u \cdot \nabla\zeta} + \beta v - f \frac{\partial w}{\partial z} = 0$$

In [8]:
file_name = os.path.join(outdir, 'qgvb.nc')
if os.path.exists(file_name):
    qgvb = xr.open_dataset(file_name)
else:

    # calculate relative vorticity
    zeta = relative_vorticity(vel_sel, grid, delta_names=('dxt', 'dyt'))
    qgvb = xr.merge([vel_sel, zeta])
    
    # # wind and bottom stress dataset
    # wind_stress = xr.merge([coordinates, taux_mm_lim, tauy_mm_lim])
    # # wind_stress = wind_stress.sel(xt_ocean=slice(-225, -211), xu_ocean=slice(-225, -211))
    # wind_stress = wind_stress.assign_coords({'xt_ocean':((wind_stress.xt_ocean + 180) % 360) - 180, 'xu_ocean':((wind_stress.xu_ocean + 180) % 360) - 180})
    
    # bottom_stress = xr.merge([coordinates, bmf_u_mm_lim, bmf_v_mm_lim])
    # # bottom_stress = bottom_stress.sel(xt_ocean=slice(-225, -211), xu_ocean=slice(-225, -211))
    # bottom_stress = bottom_stress.assign_coords({'xt_ocean':((bottom_stress.xt_ocean + 180) % 360) - 180, 'xu_ocean':((bottom_stress.xu_ocean + 180) % 360) - 180})
    
    # calculate vorticity terms
    dzetadt = vorticity_tendency(qgvb)
    ugradzeta = adv_relative_vorticity(qgvb, grid)
    betav = adv_planetary_vorticity(qgvb, grid) #, beta
    fdwdz = vortex_stretching(qgvb, grid, delta_names=('dxt', 'dyt'))
    
    # quasi-geostrophic vorticity balance
    qgvb = xr.merge([qgvb, dzetadt, ugradzeta, betav[0], fdwdz]) # betav[0] first output variable in function, see oceanpy
    
    to_netcdf(qgvb, os.path.join(outdir, 'qgvb.nc'))

## Calculate depth-integrated vorticity terms

In [9]:
qgvb

<xarray.Dataset>
Dimensions:    (xt_ocean: 144, yt_ocean: 115, st_ocean: 75, time: 30,
                xu_ocean: 144, yu_ocean: 116, sw_ocean: 75)
Coordinates:
  * xt_ocean   (xt_ocean) float64 -225.1 -225.0 -224.9 ... -211.0 -210.9 -210.8
  * yt_ocean   (yt_ocean) float64 -53.67 -53.61 -53.55 ... -46.5 -46.43 -46.36
  * st_ocean   (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time       (time) datetime64[ns] 1997-04-01T12:00:00 ... 1997-04-30T12:00:00
  * xu_ocean   (xu_ocean) float64 -225.2 -225.1 -225.0 ... -211.1 -211.0 -210.9
  * yu_ocean   (yu_ocean) float64 -53.7 -53.64 -53.58 ... -46.47 -46.4 -46.33
  * sw_ocean   (sw_ocean) float64 1.083 2.279 3.601 ... 5.61e+03 5.809e+03
Data variables: (12/15)
    dxt        (yt_ocean, xt_ocean) float32 ...
    dyt        (yt_ocean, xt_ocean) float32 ...
    dst        (st_ocean, yt_ocean, xt_ocean) float32 ...
    dxu        (yu_ocean, xu_ocean) float32 ...
    dyu        (yu_ocean, xu_ocean) float32 ...
    area_t     (yt_ocean, xt_ocean) float32 ...
    ...         ...
    wt         (time, sw_ocean, yt_ocean, xt_ocean) float32 ...
    zeta       (time, st_ocean, yu_ocean, xu_ocean) float32 ...
    dzetadt    (time, st_ocean, yu_ocean, xu_ocean) float32 ...
    ugradzeta  (time, st_ocean, yu_ocean, xu_ocean) float32 ...
    betav      (yu_ocean, xu_ocean, time, st_ocean) float64 ...
    fdwdz      (yu_ocean, xu_ocean, time, st_ocean) float64 ...
Attributes:
    long_name:     ocean dxt on t-cells
    units:         m
    valid_range:   [-1.e+09  1.e+09]
    cell_methods:  time: point
    ncfiles:       /g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra...
    contact:       Andrew Kiss
    email:         andrew.kiss@anu.edu.au
    created:       2020-06-09
    description:   0.1 degree ACCESS-OM2 global model configuration under int...
    notes:         Source code: https://github.com/COSIMA/access-om2 License:...

In [9]:
file_name = os.path.join(outdir, 'qgvb_dint.nc')
if not os.path.exists(file_name):

    # calculate vertically integrated vorticity balance
    dzetadt_dint = grid.integrate(qgvb.dzetadt, 'S')
    dzetadt_dint.name = qgvb.dzetadt.name + '_dint'
    ugradzeta_dint = grid.integrate(qgvb.ugradzeta, 'S')
    ugradzeta_dint.name = qgvb.ugradzeta.name + '_dint'
    betav_dint = grid.integrate(qgvb.betav, 'S')
    betav_dint.name = qgvb.betav.name + '_dint'
    fdwdz_dint = grid.integrate(qgvb.fdwdz, 'S')
    fdwdz_dint.name = qgvb.fdwdz.name + '_dint'
    
    # # forcing
    # vert_div_tau_w = (1/rho_0) * stress_curl(wind_stress, grid, stress_names=('tau_x', 'tau_y'))
    # # vert_div_tau_w = (1/rho_0) * grid.interp(grid.interp(vert_div_tau_w, 'X', boundary='extend'), 'Y', boundary='extend')
    # vert_div_tau_w.name = 'vertical_div_wind_stress_curl'
    # vert_div_tau_b = (1/rho_0) * stress_curl(bottom_stress, grid, stress_names=('bmf_u', 'bmf_v'))
    # vert_div_tau_b.name = 'vertical_div_bottom_stress_curl'
    
    # qgvb_forcing = xr.merge([vert_div_tau_w, vert_div_tau_b])
    # # qgvb_forcing = qgvb_forcing.sel(xu_ocean=slice(-225, -211))
    # # qgvb_forcing = qgvb_forcing.assign_coords({'xu_ocean':((qgvb_forcing.xu_ocean + 180) % 360) - 180})
    
    # # qgvb_dint = xr.merge([vel_month, zeta])
    
    qgvb_dint = xr.merge([vel_sel, dzetadt_dint, ugradzeta_dint, betav_dint, fdwdz_dint])
    to_netcdf(qgvb_dint, file_name)

## Calculate bottom pressure torque

Bottom pressure torque as explained in Jackson et al., 2006 and Stewart et al. 2021

$$BPT \equiv -J(p_b, \eta_b)$$

$$J(A,B) = \frac{\partial A}{\partial x}\frac{\partial B}{\partial y} - \frac{\partial A}{\partial y}\frac{\partial B}{\partial x}$$

Bottom pressure torque is defined as,
\begin{equation}
\begin{aligned}
-\frac{1}{\rho_0}J(p_b, h) &= -f{\bf u_{b,g}} \cdot {\bf\nabla_H} h \\
& = -\frac{1}{\rho_0} \left( \frac{\partial p_b}{\partial x} \frac{\partial h}{\partial y} - \frac{\partial p_b}{\partial y} \frac{\partial h}{\partial x} \right)
\end{aligned}
\end{equation}
where $J$ is the Jacobi operator

"bottom pressure torque can be interpreted in terms of the geostrophic flow at the sea floor"

$$\rho f {\bf u_b} = ({\bf k} \times {\bf \nabla} p)_b$$
$$ \rho f {\bf u_b} \cdot {\bf \nabla}\eta_b = J(p_b, \eta_b)$$ 

"As there is no flow through the seafloor, any horizontal flow toward an isobath must be accompanied by a vertical flow"

$$w_b = -{\bf u_b} \cdot {\bf \nabla}\eta_b$$

such that the BPT can be defined as

$$\rho f w_b = -J(p_b, \eta_b)$$

"When friction is allowed, this becomes a condition at the top of the bottom Ekman layer, and the effect of bottom stress curl must also be allowed."

Firing 2016: vortex stretching due to flow over topography
$$fw_b=f{\bf u_b}\cdot\nabla_h\eta_b$$

In [34]:
def bottom_pressure_torque(ds, grid, bottom_cells=('kmt', 'kmu'), topo='ht', rho_0=1036):

    # bottom pressure
    kmt = ds[bottom_cells[0]].astype(int) - 2
    kmt.load()
    pb = ds.pressure.isel(st_ocean=kmt)
    # pb = grid.interp(grid.interp(pb_t, 'X', boundary='extend'), 'Y', boundary='extend')

    # bottom geostrophic velocities
    kmu = ds[bottom_cells[1]].astype(int) - 2
    kmu.load()
    ubg = ds.ug.isel(st_ocean=kmu)
    vbg = ds.vg.isel(st_ocean=kmu)
    
    # topography gradients interpolated on u-cells
    dhdx = grid.interp(grid.diff(ds[topo], 'X', boundary='extend'), 'Y', boundary='extend')  / ds['dxu']
    dhdy = grid.interp(grid.diff(ds[topo], 'Y', boundary='extend'), 'X', boundary='extend')  / ds['dyu']

    # pressure gradients interpolated on u-cells
    dpbdx = grid.interp(grid.diff(pb, 'X', boundary='extend'), 'Y', boundary='extend')  / ds['dxu']
    dpbdy = grid.interp(grid.diff(pb, 'Y', boundary='extend'), 'X', boundary='extend')  / ds['dyu']

    # # Coriolis parameter
    # fcor,_ = xr.broadcast(f(coordinates.yu_ocean), coordinates.xu_ocean)
    # fcor.name = 'f'
    # fcor.attrs = {'standard_name': 'coriolis_parameter', 'units': r'$\textrm{s}^{-1}$', 'long_name': 'Coriolis parameter'}
    
    # bottom pressure torque
    bpt = - (1 / rho_0) * ((dpbdx * dhdy) - (dpbdy * dhdx))
    bpt_g = - ds.f * ( (ubg * dhdx) + (vbg * dhdy) )

    return bpt, bpt_g

In [35]:
file_name = os.path.join(outdir, 'bpt.nc')
# if not os.path.exists(file_name):

pressure_sel = hydro.pressure.sel(time=monthly_period)

# aggregate all required variables
bpt_sel = xr.merge([coordinates, pressure_sel, geos_vel.ug, geos_vel.vg, geos_vel.f], compat='override')

# calculate bottom pressure torque
bpt, bpt_g = bottom_pressure_torque(bpt_sel, grid)
bpt.name, bpt_g.name = 'bpt', 'bpt_g'
bpt = xr.merge([bpt_sel, bpt, bpt_g])

# smoothing topography
ht_smooth = coordinates.ht.copy(deep=True)
ht_smooth.values = uniform_filter(ht_smooth, (10,10))
ht_smooth.name = 'ht_smooth'

bpt_sel = xr.merge([bpt_sel, ht_smooth])
bpt_smooth, bpt_g_smooth = bottom_pressure_torque(bpt_sel, grid, topo='ht_smooth')
bpt_smooth.name, bpt_g_smooth.name = 'bpt_sm', 'bpt_g_sm'

bpt = xr.merge([bpt, ht_smooth, bpt_smooth, bpt_g_smooth])

# save file to disk
to_netcdf(bpt, file_name)


## Calculate bottom vortex stretching

In [36]:
def vortex_stretching_topography(ds, grid, topo='ht'):
    
    # Coriolis parameter
    fcor,_ = xr.broadcast(f(ds.yt_ocean), ds.xt_ocean)
    fcor.name = 'f_t'
    fcor.attrs = {'standard_name': 'coriolis_parameter', 'units': r'$\textrm{s}^{-1}$', 'long_name': 'Coriolis parameter'}

    # bottom velocities
    kmu = ds.kmu.astype(int) - 2
    kmu.load()
    u_b = ds.u.isel(st_ocean=kmu)
    v_b = ds.v.isel(st_ocean=kmu)

    kmt = ds.kmt.astype(int) - 2
    kmt.load()
    w_b = ds.wt.isel(sw_ocean=kmt)
    
    # topography gradients interpolated on u-cells
    dhdx = grid.interp(grid.diff(ds[topo], 'X', boundary='extend'), 'Y', boundary='extend') / ds.dxu
    dhdy = grid.interp(grid.diff(ds[topo], 'Y', boundary='extend'), 'X', boundary='extend') / ds.dyu

    # vortex stretching due to topography (kinematic boundary condition)
    vst = ds.f * ( (u_b * dhdx) + (v_b * dhdy) )
    fwb = fcor * w_b
    
    return vst, fwb

In [37]:
file_name = os.path.join(outdir, 'vst.nc')
# if not os.path.exists(file_name):

vst_sel = xr.merge([vel_sel, geos_vel.f, coordinates.get(['ht', 'kmt', 'kmu'])])
vst, fwb = vortex_stretching_topography(vst_sel, grid)
vst.name, fwb.name = 'vst', 'fwb'
vst = xr.merge([vst_sel, vst, fwb])

vst_sel = xr.merge([vst_sel, ht_smooth])
vst_smooth, fwb_smooth = vortex_stretching_topography(vst_sel, grid, topo='ht_smooth')
vst_smooth.name, fwb_smooth.name = 'vst_sm', 'fwb_sm'
vst = xr.merge([vst, vst_smooth, fwb_smooth])

# save file to disk
to_netcdf(vst, file_name)
    

TypeError Invalid value for attr 'c_grid_axis_shift': None. For serialization to netCDF files, its value must be of one of the following types: str, Number, ndarray, number, list, tuple
